### 1.FFM背景

&emsp;&emsp;FFM(Field-aware Factorization Machine)场感知因子分解机，是对因子分解机的升级版模型，通过引入field的概念，把相同性质的特征归于同一个field。

### 2.分析FM模型
&emsp;&emsp;先看一下FM模型的公式:
$$y(x) = w_{0} + \sum_{i=1}^{n}{w_{i}x_{i}} + \sum_{i=1}^{n}\sum_{j=i+1}^{n}{<v_{i},v_{j}>x_{i}x_{j}} \tag{1}$$

&emsp;&emsp;可以看出FM对每两个特征都进行关联分析，首先分析一下，一个类别向量one-hot之后，假设年龄sex是一个类别数据，分为man/woman，在one-hot后是两列数据，FM模型虽然会去考虑man列和woman列的交叉特征，但是$x_{man}x_{woman}==0$,因此一个类别下的特征交叉为0，

### 3.FFM算法
#### 3.1 算法简介
&emsp;&emsp;FFM引入field概念，FFM把相同性质的特征归于同一个field。在FFM中，每一维特征，针对其他特征的每一种field $f_{j}$，都会学习一个隐向量 $V_{i,f_{j}}$，因此隐向量不仅与特征相关，也与field相关，这也是FFM中"Field-aware"的由来。

#### 3.2 算法原理
&emsp;&emsp;假设样本一共有n个特征，f个field，那么FFM的二次项一共有nf个隐向量。而在FM模型中，每一维特征的隐向量只有一个，FM可以看作FFM的特例，是把所有特征都归属到一个field时的FFM模型。根据FFM的field敏感特性，可以导出其模型方程。

$$y = w_{0} + \sum_{i=1}^{n}{w_{i}x_{i}} + \sum_{i=1}^{n} \sum_{j=i+1}^{j=n}{V_{i,f_{j}},V_{j,f_{i}}x_{i}x_{j}} \tag{2}$$

&emsp;&emsp;其中$f_{j}$是第j个特征所属的field字段，如果隐向量的长度为k，那么FFM的二次参数有nfk个，远多于FM模型的nk个。此外由于隐向量与field相关，FFM二次项并不能够化简，时间复杂度是$O(kn^{2})$。

&emsp;&emsp;**需要注意的是FFM的latent vector只需要学习特定的field，因此在参数更新的时候比FM要迅速**

#### 3.3 FFM举例

| Clicked | Publisher\(P\) | Advertiser\(A\) | Gender\(G\) |
|---------|----------------|-----------------|-------------|
| Yes     | ESPN           | Nike            | Male        |

&emsp;&emsp;对于FM模型来说，其交叉项为:

$$F_{FM}(V,x) = <V_{ESPN},V_{Nike}> + <V_{ESPN},V_{Male}> + <V_{Nike},V_{Male}> \tag{3}$$

&emsp;&emsp;因为在FM中，每个特征只有一个latent vector ，这个latent 可以用来学习和其他特征之间的关系，但是在FFM中，每一个特征有好几个latent vector，取决于其他特征的字段；对于FFM模型来说，其交叉项为:

$$F_{FM}(V,x) = <V_{ESPN,A},V_{Nike,P}> + <V_{ESPN,G},V_{Male,P}> + <V_{Nike,G},V_{Male,A}> \tag{4}$$

&emsp;&emsp;简单来说，在交叉的时候需要考虑到另一个特征属于的field,假设有f个field，因此每个特征都有f个latent vector。